In [ ]:
from datasets import load_dataset, load_dataset_builder

In [ ]:
name = "fashion_mnist"
name = "wikimedia/wikipedia"
name = "daloopa/fashion-mnist-interview"
ds_builder = load_dataset_builder(name)
ds_builder.info

DatasetInfo(description='', citation='', homepage='', license='', features={'image': Image(decode=True, id=None), 'label': ClassLabel(names=['T - shirt / top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'], id=None)}, post_processed=None, supervised_keys=None, task_templates=None, builder_name='parquet', dataset_name='fashion-mnist-interview', config_name='default', version=0.0.0, splits={'train': SplitInfo(name='train', num_bytes=31049107.0, num_examples=60000, shard_lengths=None, dataset_name=None), 'test': SplitInfo(name='test', num_bytes=4150316.0, num_examples=8000, shard_lengths=None, dataset_name=None)}, download_checksums=None, download_size=33099036, post_processing_size=None, dataset_size=35199423.0, size_in_bytes=None)

In [ ]:
ds_builder.info.features

{'image': Image(decode=True, id=None),
 'label': ClassLabel(names=['T - shirt / top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'], id=None)}

In [ ]:
ds_builder.info.splits

{'train': SplitInfo(name='train', num_bytes=31049107.0, num_examples=60000, shard_lengths=None, dataset_name=None),
 'test': SplitInfo(name='test', num_bytes=4150316.0, num_examples=8000, shard_lengths=None, dataset_name=None)}

In [ ]:
dataset = load_dataset(name)

Generating train split:   0%|          | 0/60000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8000 [00:00<?, ? examples/s]

In [ ]:
train, test = dataset['train'], dataset['test']
train[0]

{'image': <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>,
 'label': 9}

In [ ]:
x,y = ds_builder.info.features # image, label
x,y

('image', 'label')

In [ ]:
xb[0]

In [ ]:
xb, yb = train[:5][x], train[:5][y]
xb, yb

([<PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>,
  <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>,
  <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>,
  <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>,
  <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>],
 [9, 0, 0, 3, 0])

In [ ]:
train.features[y].int2str(yb)

['Ankle boot',
 'T - shirt / top',
 'T - shirt / top',
 'Dress',
 'T - shirt / top']

## Collate

All prepocessing could be done in the `collate_fn`. This takes a batch as input and returns a type to be used by the model.

This could be a tuple or dict (common in HF).

pytorch has `default_collate` that works on tuple or dict

This could do all the preprocessing on the batch, image augmentations etc.

In [ ]:
import torch
from torch.utils.data import DataLoader
import torchvision.transforms.functional as TF

def collate_fn(b):
    #  [{'image': <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28 at 0x2AF1BE3B0>, 'label': 9}, {'image': <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28 at 0x2AF1BDF60>, 'label': 0},  ... ]
    return {x: torch.stack([TF.to_tensor(o[x]) for o in b]), y: torch.tensor([o[y] for o in b])}

dl = DataLoader(train, batch_size=32, collate_fn=collate_fn)
b = next(iter(dl))
b[x].shape, b[y].shape

[{'image': <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>, 'label': 9}, {'image': <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>, 'label': 0}, {'image': <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>, 'label': 0}, {'image': <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>, 'label': 3}, {'image': <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>, 'label': 0}, {'image': <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>, 'label': 2}, {'image': <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>, 'label': 7}, {'image': <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>, 'label': 2}, {'image': <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>, 'label': 5}, {'image': <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>, 'label': 5}, {'image': <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>, 'label': 0}, {'image': <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>, 'label': 9}, {'image': <PIL.

(torch.Size([32, 1, 28, 28]), torch.Size([32]))

Or we can use transforms instead:

In [ ]:
train[0]

{'image': <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>,
 'label': 9}

In [ ]:
def transforms(d):
    # notice this is working on whole datasets, rather than individual batches
    # above we indexed into 
    # {'image': [<PIL.PngImagePlugin.PngImageFile image mode=L size=28x28 at 0x2AF1BE080>, <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28 at 0x2AF1BE8F0>, ... ],
    #  'label': [9, 0, ... ]}
    
    d[x] = [TF.to_tensor(o) for o in d[x]] 
    return d # even though inplace, still have to return d

In [ ]:
tds = train.with_transform(transform=transforms) # this function is called on the fly (can be used for data aug)
dl = DataLoader(tds, batch_size=32)
b = next(iter(dl))
b[x].shape, b[y].shape

(torch.Size([32, 1, 28, 28]), torch.Size([32]))

In [ ]:
def inplace(f):
    def _inner(x):
        f(x)
        return x
    return _inner

In [ ]:
@inplace
def transfomi(d): d[x] = [TF.to_tensor(o).flatten() for o in d[x]] 

In [ ]:
itemgetter(dict(a=3, b=4), 'a')

NameError: name 'itemgetter' is not defined

In [ ]:
tds = train.with_transform(transform=transfomi) # this function is called on the fly (can be used for data aug)
dl = DataLoader(tds, batch_size=32)
b = next(iter(dl))
b[x].shape, b[y].shape

(torch.Size([32, 784]), torch.Size([32]))